In [1]:
is_it_a_test = True
test_name = "_feature"

In [2]:
#%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


In [3]:
import numpy as np
import pandas as pd

from src.data.collect_data import *
import src.data.get_dataset as get_dataset

In [4]:
vocab = initiate_vocab(test=is_it_a_test, test_name=test_name)

all_vocab = get_dataset.get_all_vocab()

historical_data, tries_session, nb_known_words_session = initiate_session()

In [7]:
for i in range(100):

    i_vocab_try = dict()

    # Choose language
    i_vocab_try = choose_a_language(i_vocab_try, vocab)
    if i_vocab_try["input_language"] == -1:
        break

    # Pick a random word
    i_vocab_try = choose_a_word(i_vocab_try, vocab)

    # Take a guess
    i_vocab_try = prompt_question(
        i_vocab_try,
        print_info={
            "nb_known_words_session": nb_known_words_session,
            "tries_session": tries_session,
        },
    )

    # Check answer
    i_vocab_try = check_answer(i_vocab_try, all_vocab)

    # Do we want to stop?
    if i_vocab_try["your_guess"] == "STOP IT":
        print(
            "Your last word was '{}={}'".format(
                i_vocab_try["word_input_language"],
                i_vocab_try["word_output_language"],
            )
        )
        break

    # If correct answer
    if i_vocab_try["is_it_correct"]:
        i_vocab_try = if_correct(i_vocab_try)
        nb_known_words_session += 1

    # If wrong answer
    if not i_vocab_try["is_it_correct"]:
        i_vocab_try = if_not_correct(i_vocab_try, vocab, all_vocab)

    # Add to historical data
    historical_data = add_historical_data(historical_data, i_vocab_try)

    # Update vocab
    vocab = update_vocab(vocab, i_vocab_try)

    tries_session += 1

0/0 - What is the german word for 'September'?   September


/Users/admin/Projects/vocabulary_learning/src/data/collect_data.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ] = softmax(1 - list_possible_words[f"{i_vocab_try['output_language']}_proba"].values.tolist())


Correct!
1/1 - What is the german word for 'married'?   scj
Sorry, it was 'married = verheiratet'
Write it again   jo
1/2 - What is the english word for 'viel'?   a lot
Correct!
2/3 - What is the german word for 'the female doctor'?   Arztin
Sorry, it was 'the female doctor = die Ärztin'
Write it again   ih
2/4 - What is the german word for 'the apricot'?   die Aprikose
Correct!
3/5 - What is the german word for 'to vote'?   aslöjcpwj
Sorry, it was 'to vote = stimmen'
Write it again   ä
3/6 - What is the german word for 'to speak'?   vkKEV+
Sorry, it was 'to speak = sprechen'
Write it again   Vo
3/7 - What is the german word for 'the year'?   vm
Sorry, it was 'the year = das Jahr'
Write it again   
3/8 - What is the english word for 'wirklich'?   
Sorry, it was 'wirklich = really'
Write it again   
3/9 - What is the english word for 'rennen'?   
Sorry, it was 'rennen = to sprint'
Write it again   
3/10 - What is the german word for 'the cup'?   
Sorry, it was 'the cup = der Pokal'
Writ

In [8]:
# Update and save historical data
update_historical_data(historical_data, test=is_it_a_test, test_name=test_name)

In [9]:
# Update and save vocab
finalize_vocab(vocab, test=is_it_a_test, test_name=test_name)

Time to add new words!! - only 14 words left'


In [10]:
# prepare next session

In [11]:
! python -m luigi --module src.luigi_pipeline PrepareNextSession --local-scheduler

DEBUG: Checking if PrepareNextSession(name=merge historical and vocab dataset features together.) is complete
INFO: Informed scheduler that task   PrepareNextSession_merge_historical_c1184df937   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=129264731, workers=1, host=Yoanns-MBP.fritz.box, username=admin, pid=59067) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 PrepareNextSession(name=merge historical and vocab dataset features together.)

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====

